## Dantzig's algorithm for parameters initialization given the Earth Mover's Distance (Wasserstein distance) model.

Russell, E. J. (1969). Extension of Dantzig’s Algorithm to Finding an Initial Near-Optimal Basis for the Transportation Problem. Operations Research, 17(1), 187–191. http://www.jstor.org/stable/168855

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def dantzig(cost,a,b):
    initial=np.zeros_like(costs)
    c=0
    col, row=[],[]
    while np.any(costs) and c!=np.product(costs.shape):
        w=[]
        y=[]
        #step 1
        for i in range(len(costs)):
            y.append(np.max(costs[:,i]))
            w.append(np.max(costs[i]))
        #step 2
        s2=np.zeros_like(costs)
        for i in range(costs.shape[0]):
            for j in range(costs.shape[1]):
                s2[i][j]=w[i]+y[j]-costs[i][j]
        #clean up the s2
        for i in range(len(col)):
            s2[:,col[i]]=np.zeros_like(s2[:,col[i]])
        for i in range(len(row)):
            s2[row[i]]=np.zeros_like(s2[row[i]])
        #step 3
        xdij=np.max(s2)
        coord=np.where(s2==xdij)
        if a[coord[0][0]]>=b[coord[1][0]]: #if row is higher than column a>b
            value=b[coord[1][0]]
            costs[:,coord[1][0]]=np.zeros_like(costs[:,coord[1][0]])# "delete" column
            col.append(coord[1][0])
            b[coord[1][0]]=0
            a[coord[0][0]]=a[coord[0][0]]-value
        else: # b>=a
            value=a[coord[0][0]]
            costs[coord[0][0]]=np.zeros_like(costs[coord[0][0]])#"delete" row
            row.append(coord[0][0])
            a[coord[0][0]]=0
            b[coord[1][0]]=b[coord[1][0]]-value
        initial[coord[0][0]][coord[1][0]]=value
        c=c+1
    return initial

In [3]:
costs=np.array([[73,40,9,79,20],[62,93,96,8,13],[96,65,80,50,65],[57,58,29,12,87],[56,23,87,18,12]])
a=[8,7,9,3,5] #origin
b=[6,8,10,4,4] #destination
result=dantzig(costs,a,b)

## Optimizing the initialized parameters from the Dantzing's algorithm

In [4]:
def objective(x):
    minim=0
    for i in range(costs.shape[0]):
        for j in range(costs.shape[1]):
            minim=minim+costs[i][j]*x[i*(5)+j]
            #print(i,j,costs[i][j],x[i*(5)+j], costs[i][j]*x[i*(4)+j])
    return minim
def constraint0(x):
    a0m=a[0]
    for j in range(costs.shape[0]):
        a0m=a0m-x[0*(len(costs))+j]
    return a0m
def constraint1(x):
    a1m=a[1]
    for j in range(costs.shape[0]):
        a1m=a1m-x[1*(len(costs))+j]
    return a1m
def constraint2(x):
    a2m=a[2]
    for j in range(costs.shape[0]):
        a2m=a2m-x[2*(len(costs))+j]
    return a2m
def constraint3(x):
    a3m=a[3]
    for j in range(costs.shape[0]):
        a3m=a3m-x[3*(len(costs))+j]
    return a3m
def constraint4(x):
    a4m=a[4]
    for j in range(costs.shape[0]):
        a4m=a4m-x[4*(len(costs))+j]
    return a4m

def constraint5(x):
    b0m=b[0]
    for j in range(costs.shape[0]):
        b0m=b0m-x[(j*5)]
    return b0m
def constraint6(x):
    b1m=b[1]
    for j in range(costs.shape[0]):
        b1m=b1m-x[(j*5+1)]
    return b1m
def constraint7(x):
    b2m=b[2]
    for j in range(costs.shape[0]):
        b2m=b2m-x[(j*5+2)]
    return b2m
def constraint8(x):
    b3m=b[3]
    for j in range(costs.shape[0]):
        b3m=b3m-x[(j*5+3)]
    return b3m
def constraint9(x):
    b4m=b[4]
    for j in range(costs.shape[0]):
        b4m=b4m-x[(j*5+4)]
    return b4m

In [5]:
costs=np.array([[73,40,9,79,20],[62,93,96,8,13],[96,65,80,50,65],[57,58,29,12,87],[56,23,87,18,12]])
a=[8,7,9,3,5] #origin
b=[6,8,10,4,4] #destination
x0=result.flatten()
print(objective(x0))

1103


In [6]:
bn=(0, None)
bnds=(bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn,bn)
con0={'type':'eq','fun':constraint0}
con1={'type':'eq','fun':constraint1}
con2={'type':'eq','fun':constraint2}
con3={'type':'eq','fun':constraint3}
con4={'type':'eq','fun':constraint4}
con5={'type':'eq','fun':constraint5}
con6={'type':'eq','fun':constraint6}
con7={'type':'eq','fun':constraint7}
con8={'type':'eq','fun':constraint8}
con9={'type':'eq','fun':constraint9}
cons=[con0,con1,con2,con3,con4,con5,con6,con7,con8,con9]

In [7]:
sol=minimize(objective,x0,method='SLSQP', bounds=bnds, constraints=cons)
test=np.round_(sol.x, decimals = 3)
print(objective(test))
test=np.reshape(test, (5,5))
test

1103.0


array([[0., 0., 8., 0., 0.],
       [0., 0., 0., 3., 4.],
       [5., 3., 0., 1., 0.],
       [1., 0., 2., 0., 0.],
       [0., 5., 0., 0., 0.]])